# José Luis Padilla
# Modulo 3 - Unidad 4 Caso práctico: Carga y limpieza de datos (datos de accidentes de Madrid)

In [1]:
import pandas as pd
import numpy as np
# Altair es una librería de visualización de datos, basada en Vega y Vega-Lite, que usa un lenguaje declarativo que busca minimizar la cantidad de código requerido para la construcción de gráficos.
import altair as alt
# El módulo "os.path" nos permite gestionar diferentes opciones relativas al sistema de ficheros como pueden ser ficheros, directorios, etc.
import os.path
# "urllib. request" es un módulo Python para acceder y utilizar recursos de internet identificados por URLs. 
# Ofrece una interfaz muy simple, a través de la función urlopen. Esta función es capaz de acceder a URLs usando una variedad de protocolos diferentes.
import urllib.request

import os

## Descripción del caso

En este caso práctico se accede a unos datos públicos y se realiza un análisis exploratorio y una limpieza y preparación de los datos.
Los datos a analizar corresponden a los accidentes de tráfico en la ciudad de Madrid, que están disponibles a través del portal de datos abiertos del Ayuntamiento de Madrid. Se trata de un conjunto de datos con múltiples dimensiones de distinto tipo y una cardinalidad pequeña, de modo que son sencillos de manipular pero ricos.

## Descarga de los datos

En primer lugar se muestra como descargar los datos y los almacenarlos localmente. Este paso no es necesario repetirlo cada vez que se ejecute este código, de manera que primero comprobamos si ya existen los ficheros para no volver a descargarlos.

(En este ejemplo, este medio de acceso a los datos está comentado, y se sustituye por un acceso local, para que sea posible ejecutar el notebook aún sin conexión a internet o aunque el sitio datos.madrid.es deje de estar disponible. De todos modos se muestra aquí con fines didácticos)

In [2]:
# Si no existe el directorio 'datos', lo creamos

# if not os.path.exists('./datos'):
#     os.makedirs('./datos')

# # URLs de los datos a acceder y nombre de los ficheros en los que guardarlos
# URL_DATOS_ACCIDENTES = 'https://datos.madrid.es/egob/catalogo/300228-19-accidentes-trafico-detalle.csv'
# PATH_DATOS_ACCIDENTES = './datos/accidentes2019.csv'

# # El ayuntamiento ofrece un pequeño documento con la descripción de los datos, que aprovechamos para descargar también
# # (Desde Jupyter Lab, basta con hacer doble click sobre el nombre del fichero en el panel de la izquierda 
# # para que se abra en una nueva pestaña)
# URL_PDF_ACCIDENTES = 'https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_DS_Accidentes_trafico_desde_2019.pdf'
# PATH_PDF_ACCIDENTES = './datos/accidentes2019.pdf'

# if not os.path.exists(PATH_DATOS_ACCIDENTES):
#     urllib.request.urlretrieve(URL_DATOS_ACCIDENTES, PATH_DATOS_ACCIDENTES)
    
# if not os.path.exists(PATH_PDF_ACCIDENTES):
#     urllib.request.urlretrieve(URL_PDF_ACCIDENTES, PATH_PDF_ACCIDENTES)

## Lectura de los datos

Una vez descargados los datos, que en este caso están en formato csv, lo más sencillo es cargarlos en un DataFrame de Pandas para proceder a una exploración inicial. En este caso concreto, para que esa carga funcione correctamente es necesario especificar el encoding del fichero csv, ya que no utiliza el estándar habitual UTF-8 pero contiene caracteres especiales ("especiales" para el estándar ASCII, como puede ser la letra eñe o las vocales acentuadas)

En primer lugar, vemos las primeras líneas del fichero, para tener una idea del contenido

In [3]:
with open('C:\\Users\\jlpad\\Desktop\\BBDD\\accidentes2019.csv', encoding = 'latin1') as f:
    for _ in range(5) : print(f.readline())

Nº  EXPEDIENTE;FECHA;HORA;CALLE;NÚMERO;DISTRITO;TIPO ACCIDENTE;ESTADO METEREOLÓGICO;TIPO VEHÍCULO;TIPO PERSONA;RANGO EDAD;SEXO;LESIVIDAD*;* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.

2019S000020;01/01/2019;23:30;CALL. FUENCARRAL;149;CHAMBERÍ;Caída;Despejado;Ciclomotor;Conductor;DE 25 A 29 AÑOS;Hombre;01;

2019S000017;01/01/2019;22:15;CALL. OCA / CALL. PINZON;-;CARABANCHEL;Colisión fronto-lateral;Despejado;Turismo;Conductor;DE 40 A 44 AÑOS;Mujer;14;

2019S000017;01/01/2019;22:15;CALL. OCA / CALL. PINZON;-;CARABANCHEL;Colisión fronto-lateral;Despejado;Ciclomotor;Conductor;DE 35 A 39 AÑOS;Hombre;03;

2019S001812;01/01/2019;21:40;CALL. BAILEN / CUSTA. SAN VICENTE;-;CENTRO;Colisión fronto-lateral;Despejado;Turismo;Conductor;DE 40 A 44 AÑOS;Hombre;14;



De esta inspección inicial podemos extraer algunas conclusiones (que también podrían obtenerse mediante ensayo y error con pd.read_csv():
- El separador de campos es el caracter ; (esto es habitual en ficheros csv en España, donde la coma se usa para separar la parte decimal de los números)
- La primera línea del fichero contiene los nombres de los campos, como es habitual en muchos ficheros csv
- El segundo campo, "FECHA", es un dato de tipo fecha, y el tercero, "HORA" es una hora del día
- Hay un comentario al final de la primera fila, que se interpretaría como el nombre de una columna más de datos, que habría que eliminar

A continuación, pasamos a leer los datos en bruto en un dataframe de Pandas, para poder procesarlos más cómodamente, y mostramos las primeras cinco filas de datos y una descripción general del contenido del dataframe

In [4]:
accidentes_raw = pd.read_csv('C:\\Users\\jlpad\\Desktop\\BBDD\\accidentes2019.csv', encoding = 'latin1', sep = ';')
accidentes_raw.head(6)

,Nº EXPEDIENTE,FECHA,HORA,CALLE,NÚMERO,DISTRITO,TIPO ACCIDENTE,ESTADO METEREOLÓGICO,TIPO VEHÍCULO,TIPO PERSONA,RANGO EDAD,SEXO,LESIVIDAD*,* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.
0,2019S000020,01/01/2019,23:30,CALL. FUENCARRAL,149,CHAMBERÍ,Caída,Despejado,Ciclomotor,Conductor,DE 25 A 29 AÑOS,Hombre,1.0,NaN
1,2019S000017,01/01/2019,22:15,CALL. OCA / CALL. PINZON,-,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 40 A 44 AÑOS,Mujer,14.0,NaN
2,2019S000017,01/01/2019,22:15,CALL. OCA / CALL. PINZON,-,CARABANCHEL,Colisión fronto-lateral,Despejado,Ciclomotor,Conductor,DE 35 A 39 AÑOS,Hombre,3.0,NaN
3,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 40 A 44 AÑOS,Hombre,14.0,NaN
4,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 30 A 34 AÑOS,Mujer,7.0,NaN
5,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Pasajero,DE 18 A 20 AÑOS,Mujer,14.0,NaN


In [5]:
accidentes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51806 entries, 0 to 51805
Data columns (total 14 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   Nº  EXPEDIENTE                                                                           51806 non-null  object 
 1   FECHA                                                                                    51806 non-null  object 
 2   HORA                                                                                     51806 non-null  object 
 3   CALLE                                                                                    51806 non-null  object 
 4   NÚMERO                                                                                   51802 non-null  object 
 5   DISTRITO                                                    

Efectivamente, vemos que ha aparecido una última columna con todos los valores NaN ("Not a Number") que podemos ignorar.

## Limpieza de datos

Vamos a analizar y limpiar columna por columna, ya que vemos que cada una es un caso distinto. Para facilitarlo, escribimos una pequeña función que nos proporciona los datos más relevantes para esa limpieza

In [6]:
def describe_columna(df, col):
    print(f'Columna: {col}  -  Tipo de datos: {df[col].dtype}')
    print(f'Número de valores nulos: {df[col].isnull().sum()}  -  Número de valores distintos: {df[col].nunique()}')
    print('Valores más frecuentes:')
    for i, v in df[col].value_counts().iloc[:10].items() :
        print(i, '\t', v)

In [7]:
describe_columna(accidentes_raw, 'Nº  EXPEDIENTE')

Columna: Nº  EXPEDIENTE  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 21935
Valores más frecuentes:
2019S036555 	 13
2019S028162 	 12
2019S013324 	 12
2019S000625 	 12
2019S038336 	 12
2019S029891 	 12
2019S032214 	 12
2019S024660 	 11
2019S022408 	 11
2019S017281 	 11


Nada particulamente interesante en esta columna. Vemos que el número de expediente no es único (el valor más usado, el primero de la lista de valores, se usa un total de 13 veces): leyendo el documento de descripción de los datos, o simplemente inspeccionando los datos, podemos ver que el número de expediente identifica un accidente, pero en él pueden haber estado involucrados varios vehículos o más de una persona, y para cada uno de ellos existe una línea en este fichero. El número de accidentes, por lo tanto, no es el número de filas del fichero, sino el número de valores únicos del campo Nº  EXPEDIENTE.

Para la limpieza campo por campo, VAMOS A CREAR UN NUEVO DATA FRAME al que asignaremos cada columna ya procesada. En este caso, el UNICO CAMBIO que realizamos es CORREGIR EL NOMBRE DEL CAMPO para que no tenga doble espacio entre 'Nº' y 'EXPEDIENTE', y pasar el nombre a minúsculas.

In [8]:
accidentes = pd.DataFrame()
accidentes['Nº Expediente'] = accidentes_raw['Nº  EXPEDIENTE']

Siguiente columna...

In [9]:
describe_columna(accidentes_raw, 'FECHA')
print()
describe_columna(accidentes_raw, 'HORA')

Columna: FECHA  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 365
Valores más frecuentes:
05/04/2019 	 272
31/05/2019 	 239
24/04/2019 	 234
01/02/2019 	 229
20/12/2019 	 229
12/12/2019 	 226
22/02/2019 	 212
19/01/2019 	 210
04/10/2019 	 209
31/10/2019 	 207

Columna: HORA  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 1213
Valores más frecuentes:
18:00 	 464
19:30 	 457
20:00 	 452
16:00 	 452
18:30 	 441
17:00 	 424
13:00 	 421
21:00 	 405
14:00 	 395
14:30 	 395


Vemos que tanto las FECHAS como las HORAS se han leido como TIPO de datos "OBJECT", que es la forma como Pandas identifica las cadenas de caracteres (al menos, hasta la versión 1.0, a partir de la cual existe un tipo específico, string, para las cadenas de caracteres). Para que sea más sencillo operar con ellas, LAS TRANSFORMAMOS a TIPO de datos "DATETIME" Y LAS JUNTAMOS en 1 SOLO CAMPO QUE INCLUYA tanto la FECHA como la HORA (para poder ordenar los accidentes, buscar fácilmente todos los accidentes entre dos instantes concretos cualquiera, etc.)

In [10]:
accidentes['Fecha'] = pd.to_datetime(accidentes_raw.FECHA) + \
                      pd.to_timedelta(accidentes_raw.HORA + ':00')

Siguientes columnas...

In [11]:
describe_columna(accidentes_raw, 'CALLE')

Columna: CALLE  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 10456
Valores más frecuentes:
PASEO. CASTELLANA 	 719
CALL. ALCALA 	 646
CALL. BRAVO MURILLO 	 265
AVDA. ALBUFERA 	 264
CALL. FRANCISCO SILVELA 	 179
CALL. SERRANO 	 177
PASEO. SANTA MARIA DE LA CABEZA 	 169
PASEO. EXTREMADURA 	 167
CALL. PRINCIPE DE VERGARA 	 167
CALL. DOCTOR ESQUERDO 	 153


In [12]:
accidentes['Calle'] = accidentes_raw['CALLE']

In [13]:
describe_columna(accidentes_raw, 'NÚMERO')

Columna: NÚMERO  -  Tipo de datos: object
Número de valores nulos: 4  -  Número de valores distintos: 604
Valores más frecuentes:
- 	 28523
1 	 1489
2 	 1077
3 	 701
4 	 642
0 	 519
7 	 494
5 	 448
8 	 421
6 	 388


Podría parecer una buena idea convertir el campo "NÚMERO" a un tipo numérico, pero comprobemos primero cuantos valores se podrían convertir...

In [14]:
# isnumeric() nos indica si un valor se puede trasformar en un tipo numérico o no
accidentes_raw['NÚMERO'].str.isnumeric().value_counts()

False    29306
True     22496
Name: NÚMERO, dtype: int64

29.306 valores no se pueden convertir a números, mas del 50% del total. Comprobemos qué contienen esos campos

In [15]:
# Seleccionamos los registros de accidentes_raw para los que isnumeric() del campo "NÚMERO" es falso,
# nos quedamos con ese campo, y contamos los valores que más se repiten
accidentes_raw[accidentes_raw['NÚMERO'].str.isnumeric() == False]['NÚMERO'].value_counts()

-        28523
 1A         68
 5A         35
 2A         30
 259A       19
         ...  
 200A        1
2O           1
 T-3         1
 7D          1
 8B          1
Name: NÚMERO, Length: 161, dtype: int64

Vemos que, aunque el campo se llama "NÚMERO", contiene valores que son cadenas de texto. También vemos ahí que hay un valor muy frecuente, el caracter "-", que se utiliza para indicar que no hay información disponible (no tiene ningún significado adicional). Siempre es conveniente normalizar los valores correspondientes a la falta de datos, y en este caso, la mejor manera de hacerlo es utilizar el mecanismo estándar de Pandas para marcar datos faltantes, los valores NaN

In [16]:
accidentes['Número'] = accidentes_raw['NÚMERO']
accidentes.loc[accidentes['Número'] == '-', 'Número'] = np.NaN

Siguiente campo...

In [17]:
describe_columna(accidentes_raw, 'DISTRITO')

Columna: DISTRITO  -  Tipo de datos: object
Número de valores nulos: 5  -  Número de valores distintos: 21
Valores más frecuentes:
SALAMANCA 	 4078
PUENTE DE VALLECAS 	 4063
CHAMARTÍN 	 3891
CARABANCHEL 	 3365
CIUDAD LINEAL 	 3362
SAN BLAS-CANILLEJAS 	 2797
CENTRO 	 2752
MONCLOA-ARAVACA 	 2688
CHAMBERÍ 	 2680
RETIRO 	 2671


Como el número de valores únicos de este campo es pequeño, y además los valores posibles están limitados (no hay en Madrid más distritos que esos, cualquier otro valor no debería aceptarse), conviene convertir el valor a tipo "category", ya que esto reduce el almacenamiento necesario, acelera los cálculos y facilita el análisis

In [18]:
accidentes['Distrito'] = accidentes_raw['DISTRITO'].astype('category')

Los siguientes cinco campos presentan casos similares al caso de "DISTRITO", y se tratan de la misma manera

In [19]:
for col in accidentes_raw.columns[7:12].to_list():
    describe_columna(accidentes_raw, col)
    print()

Columna: ESTADO METEREOLÓGICO  -  Tipo de datos: object
Número de valores nulos: 5132  -  Número de valores distintos: 7
Valores más frecuentes:
Despejado 	 39984
Lluvia débil 	 3465
Nublado 	 1862
Se desconoce 	 695
LLuvia intensa 	 652
Granizando 	 14
Nevando 	 2

Columna: TIPO VEHÍCULO  -  Tipo de datos: object
Número de valores nulos: 178  -  Número de valores distintos: 29
Valores más frecuentes:
Turismo 	 36499
Motocicleta > 125cc 	 3527
Furgoneta 	 3125
Motocicleta hasta 125cc 	 2529
Autobús 	 1408
Camión rígido 	 1167
Bicicleta 	 884
Ciclomotor 	 809
Todo terreno 	 689
Otros vehículos con motor 	 330

Columna: TIPO PERSONA  -  Tipo de datos: object
Número de valores nulos: 25  -  Número de valores distintos: 3
Valores más frecuentes:
Conductor 	 41604
Pasajero 	 8357
Peatón 	 1820

Columna: RANGO EDAD  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 18
Valores más frecuentes:
DE 40 A 44 AÑOS 	 5736
DE 25 A 29 AÑOS 	 5610
DE 35 A 39 AÑOS 	 55

In [20]:
for col in accidentes_raw.columns[7:12].to_list():
    accidentes[col.capitalize()] = accidentes_raw[col].astype('category')

accidentes.loc[accidentes['Estado metereológico'] == 'Se desconoce', 'Estado metereológico'] = np.NaN

accidentes.loc[accidentes['Rango edad'] == 'DESCONOCIDA', 'Rango edad'] = np.NaN

Un detalle sobre el campo "Rango edad": cuando los valores de un campo categórico tienen un orden natural, como en este caso, conviene representarlos mediante un tipo de datos categórico ordenado, porque esto facilita el análisis, permite hacer intervalos, hace que los gráficos salgan ordenados automáticamente, etc. En este caso, el orden automático que se asigna a las categorías, que es el alfabético, no coincide con el orden real, así que hay que asignarlo manualmente

In [21]:
rangos_edad = pd.CategoricalDtype(
    ['DE 0 A 5 AÑOS', 'DE 6 A 9 AÑOS', 'DE 10 A 14 AÑOS', 'DE 15 A 17 AÑOS',
     'DE 18 A 20 AÑOS', 'DE 21 A 24 AÑOS', 'DE 25 A 29 AÑOS', 'DE 30 A 34 AÑOS', 
     'DE 35 A 39 AÑOS', 'DE 40 A 44 AÑOS', 'DE 45 A 49 AÑOS', 'DE 50 A 54 AÑOS',
     'DE 55 A 59 AÑOS', 'DE 60 A 64 AÑOS', 'DE 65 A 69 AÑOS', 'DE 70 A 74 AÑOS', 
     'MAYOR DE 74 AÑOS'],
    ordered=True)

accidentes['Rango edad'] = accidentes['Rango edad'].astype(rangos_edad)

Última columna...

In [22]:
describe_columna(accidentes_raw, 'LESIVIDAD*')

Columna: LESIVIDAD*  -  Tipo de datos: float64
Número de valores nulos: 21776  -  Número de valores distintos: 9
Valores más frecuentes:
14.0 	 16599
7.0 	 7110
2.0 	 2157
6.0 	 1555
1.0 	 1320
5.0 	 715
3.0 	 539
4.0 	 34
77.0 	 1


El campo "LESIVIDAD*" presenta un caso interesante: los valores están codificados, y la clave para entenderlos está en el documento de explicación de los datos. Allí se describen los valores de esta manera:

LESIVIDAD | Descripción
--- | --- 
01        |  Atención en urgencias sin posterior ingreso. - LEVE
02        |  Ingreso inferior o igual a 24 horas - LEVE 
03        |  Ingreso superior a 24 horas. - GRAVE
04        |  Fallecido 24 horas - FALLECIDO
05        |  Asistencia sanitaria ambulatoria con posterioridad - LEVE
06        |  Asistencia sanitaria inmediata en centro de salud o mutua - LEVE
07        |  Asistencia sanitaria sólo en el lugar del accidente - LEVE
14        |  Sin asistencia sanitaria
77        |  Se desconoce
En blanco |  Sin asistencia sanitaria

Para normalizar, en primer lugar podemos asignar los valores en blanco (leidos con NaN en accidentes_raw) a la categoría 14, y los valores de la categoría 77 asignarlos como NaN, que es la manera de representar datos faltantes o desconocidos. Adicionalmente, para tener accesible la información de la gravedad de las lesiones, que está codificada de manera oculta en las distintas categorías de lesividad, podemos crear un campo adicional con esos valores, que además puede ser una categoría ordenada con las ventajas que eso tiene.

In [23]:
accidentes['Lesividad'] = accidentes_raw['LESIVIDAD*']

In [24]:
accidentes.loc[accidentes['Lesividad'].isnull(), 'Lesividad']  = 14
accidentes.loc[accidentes['Lesividad'] == 77, 'Lesividad'] = np.NaN

In [25]:
# Definición del tipo categórico para los grados de gravedad
grados_gravedad = pd.CategoricalDtype(['Ileso', 'Leve', 'Grave', 'Fallecido'], ordered=True)

# Definimos un diccionario que mapee los valores del Lesividad con los de Gravedad
# Por comodidad, no incluimos los valores correspondientes a 'Leve', sino que se utiliza ese como valor por defecto
dict_gravedad = {14: 'Ileso', 3: 'Grave', 4: 'Fallecido'} 

# Generamos un nuevo campo 'Gavedad' calculando el valor que corresponde al grado de Lesividad
# de cada caso, y convirtiendolo al tipo categórico ordenado 'grados_gravedad'
accidentes['Gravedad'] = accidentes['Lesividad'].apply(
    lambda x: dict_gravedad.get(x,'Leve')).astype(grados_gravedad)

## Comprobación y análisis

Una vez limpiados los datos, podemos comprobar el dataframe generado...

In [26]:
accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51806 entries, 0 to 51805
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Nº Expediente         51806 non-null  object        
 1   Fecha                 51806 non-null  datetime64[ns]
 2   Calle                 51806 non-null  object        
 3   Número                23279 non-null  object        
 4   Distrito              51801 non-null  category      
 5   Estado metereológico  45979 non-null  category      
 6   Tipo vehículo         51628 non-null  category      
 7   Tipo persona          51781 non-null  category      
 8   Rango edad            46473 non-null  category      
 9   Sexo                  46756 non-null  category      
 10  Lesividad             51805 non-null  float64       
 11  Gravedad              51806 non-null  category      
dtypes: category(7), datetime64[ns](1), float64(1), object(3)
memory usage: 2.3

... y proceder al análisis exploratorio (por ejemplo, haciendo sumarizaciones o realizando gráficos que nos permitan entender mejor los datos)

In [27]:
top10_vehiculos = accidentes['Tipo vehículo'].value_counts()[:10].index.tolist()

In [28]:
# Número total de accidentados por tipo vehículo y rol del accidentado
# Agrupamos los registros por los campos "Tipo vehículo" y "Tipo persona", seleccionamos el campo "Nº Expediente",
# contamos el número de registros en cada grupo, ponemos los datos en forma de tabla cruzada y filtramos la filas
# para seleccionar solo las que corresponden a los 10 tipos de vehículo más frecuentes

accidentes.groupby(['Tipo vehículo', 'Tipo persona'])['Nº Expediente'].count().unstack().loc[top10_vehiculos]

Tipo persona,Conductor,Pasajero,Peatón
Tipo vehículo,,,
Turismo,28523,6734,1224
Motocicleta > 125cc,3179,232,115
Furgoneta,2616,371,135
Motocicleta hasta 125cc,2282,166,80
Autobús,893,467,47
Camión rígido,1040,98,29
Bicicleta,800,5,79
Ciclomotor,726,64,18
Todo terreno,539,120,30


In [29]:
# Porcentaje de accidentados por tipo vehículo y sexo
# Primero hacemos una selección similar a la del punto anterior, y a continuación, mediate la función apply,
# calculamos la división de cada valor por la suma de valores de esa fila, y lo mostramos formateado como porcentaje

acc_por_sexo = accidentes.groupby(['Tipo vehículo', 'Sexo'])['Nº Expediente'].count().unstack().loc[top10_vehiculos]

acc_por_sexo.apply(lambda x: x/x.sum(), axis=1).style.format("{:.1%}")

Sexo,Hombre,Mujer
Tipo vehículo,,
Turismo,61.5%,38.5%
Motocicleta > 125cc,84.0%,16.0%
Furgoneta,85.7%,14.3%
Motocicleta hasta 125cc,77.0%,23.0%
Autobús,69.1%,30.9%
Camión rígido,94.6%,5.4%
Bicicleta,74.3%,25.7%
Ciclomotor,75.0%,25.0%
Todo terreno,66.6%,33.4%


In [30]:
# Accidentados por sexo y rango de edad
# Generamos los datos de la siguiente manera: agrupamos por rango de edad y sexo, seleccionamos el campo "Nº Expediente",
# contamos el número de registros y reseteamos el índice 
# (esto último, para que los datos de "Rango de edad" y "Sexo" aparezcan como columnas del dataset, no como niveles del índice)
data = accidentes.groupby(['Rango edad', 'Sexo'])['Nº Expediente'].count().reset_index()

# Luego mostramos los datos en un trellis de gráficos de barras
alt.Chart(data, width=600, height=200, title='Accidentes en Madrid durante 2019').mark_bar().encode(
   x = alt.X('Rango edad:O', sort = rangos_edad.categories.to_list(), title = None),
   y = alt.Y('Nº Expediente:Q', title = 'Numero de implicados'),
   facet=alt.Facet('Sexo:N', title = None))

alt.Chart(...)

In [31]:
# Accidentados por distrito
# Generamos los datos de la siguiente manera: agrupamos por distrito, seleccionamos el campo "Nº Expediente",
# contamos el número de registros únicos (para contar accidentes, no personas accidentadas) y reseteamos el índice 
# Luego mostramos los datos en un gráfico de barras sencillo (ordenando la coordenada X, o sea, los distritos,
# en sentido inverso al valor de la coordenada Y, o sea, el número de accidentes)

data = accidentes.groupby(['Distrito'])['Nº Expediente'].nunique().reset_index()

alt.Chart(data, width=600, height=300, title='Accidentes en Madrid durante 2019').mark_bar().encode(
    x = alt.X('Distrito:N', sort = '-y', title = None),
    y = alt.Y('sum(Nº Expediente):Q', title = 'Numero de accidentes'))

alt.Chart(...)